In [ ]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

import numpy as np
import pandas as pd
import pickle as pk
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, SeparableConv1D, Dropout, MaxPooling1D, Flatten
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import cohen_kappa_score, roc_auc_score

In [ ]:
FOLDS = 5
PADDING = 100
LAYERS = 1
MAX_POWER = 10
KERNEL_SIZE = 3
POOL_SIZE = 2
DROPOUT = 0.25
ACTIVATION = 'tanh'
LAST_ACTIVATION = 'sigmoid'

input_data = pk.load(open('input_data.pkl', 'rb'))
target_data = pk.load(open('target_data.pkl', 'rb'))

layers = []
dimention = []
auc = []
kappa = []

skf = StratifiedKFold(n_splits=FOLDS, shuffle=True)
mms = MinMaxScaler()

for lay in range(1,LAYERS+1):
    for dim in np.power(2, np.arange(MAX_POWER+1)):
        print(f'layers: {lay}, nodes: {dim}')
        aucs = []
        kappas = []
        
        # Prepare training batches
        model_input = []
        model_target = []
        for i, t in zip(input_data, target_data):
            model_input.append(np.concatenate([np.zeros((PADDING - 1, i.shape[1])), i])[:PADDING,:])
            model_target.append(np.argmax(t))
        model_input = np.array(model_input)
        model_target = np.array(model_target)
        
        # Prepare k-fold training and test sets
        for train_index, test_index in skf.split(model_input, model_target):
            model_target[train_index.astype(int)]
            X_train_raw, X_test_raw = model_input[train_index], model_input[test_index]
            y_train_raw, y_test_raw = model_target[train_index], model_target[test_index]
            
            X_train = np.stack(X_train_raw)
            X_test = np.stack(X_test_raw)
            
            y_train = np.zeros((y_train_raw.size, y_train_raw.max() + 1))
            y_train[np.arange(y_train_raw.size),y_train_raw] = 1
            y_test = np.zeros((y_test_raw.size, y_test_raw.max() + 1))
            y_test[np.arange(y_test_raw.size),y_test_raw] = 1
            
            # Make the LSTM
            model = Sequential()
            model.add(SeparableConv1D(filters=dim, kernel_size=KERNEL_SIZE, activation=ACTIVATION, input_shape=(PADDING, model_input[0].shape[1])))
            for l in range(lay-1):
                model.add(SeparableConv1D(filters=dim, kernel_size=KERNEL_SIZE, activation=ACTIVATION))
                model.add(Dropout(DROPOUT))
                model.add(MaxPooling1D(pool_size=POOL_SIZE))
            model.add(Flatten())
            model.add(Dense(dim, activation=ACTIVATION))
            model.add(Dense(target_data[0].size, activation=LAST_ACTIVATION))
            model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
            es = [EarlyStopping(monitor='val_loss', patience=3, min_delta=0, restore_best_weights=True)]
            model.fit(X_train, y_train, epochs=1000, validation_split=0.25, callbacks=es, verbose=True)
            
            # Get the average auc and kappa for all affects and folds
            y_pred = model.predict(X_test, batch_size=1)
            for y_t, y_p in zip(y_test.T, y_pred.T):
                y_p = mms.fit_transform(y_p.reshape(-1, 1))
                aucs.append(roc_auc_score(y_t, y_p))
                kappas.append(cohen_kappa_score(y_t, np.around(y_p)))
        
        layers.append(lay)
        dimention.append(dim)
        auc.append(np.mean(aucs))
        kappa.append(np.mean(kappas))
        print(f'auc: {auc[-1]}')
        print(f'kappa: {kappa[-1]}')

In [ ]:
plt.figure()
for i in range(LAYERS):
    plt.plot(dimention[i*(MAX_POWER+1):i*(MAX_POWER+1)+MAX_POWER+1], auc[i*(MAX_POWER+1):i*(MAX_POWER+1)+MAX_POWER+1], marker='.', label=f'{i+1} Layer LSTM')
plt.xlabel('Projected Dimensions')
plt.ylabel('ROC AUC')
plt.title('1dCNN')
plt.legend()
plt.show()

plt.figure()
for i in range(LAYERS):
    plt.plot(dimention[i*(MAX_POWER+1):i*(MAX_POWER+1)+MAX_POWER+1], kappa[i*(MAX_POWER+1):i*(MAX_POWER+1)+MAX_POWER+1], marker='.', label=f'{i+1} Layer LSTM')
plt.xlabel('1dCNN w/ scaled softmax output')
plt.ylabel('Cohen\'s Kappa')
plt.title('1dCNN')
plt.legend()
plt.show()